### 네이버증권 페이지 별 일별 시세 크롤링 함수

In [15]:
# 검색을 통해 종목 코드와 종목 이름을 찾기
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'}

result = []
query = "WON" # 네이버증권 검색어

for item_page_no in range(1, 10): # 검색 페이지 수
    url = f'https://finance.naver.com/search/search.naver?query={query}&endUrl=&page={item_page_no}'
    response = requests.get(url, headers = headers)
    html = bs(response.text)
    a_tags = html.select('a[href*="/item/main.naver?code="]')

    for a in a_tags:
        href = a['href']
        code = href.split('=')[1]  # code= 뒤의 숫자 추출
        name = a.text  # a 태그 내부의 텍스트
        result.append((code, name))



In [16]:
def daily_market_price(item_code, page_no):

  url = f'https://finance.naver.com/item/sise_day.naver?code={item_code}&page={page_no}'

  headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'}

  response = requests.get(url, headers = headers)
  html = bs(response.text)
  table = html.select('table')
  temp = pd.read_html(str(table))
  temp = temp[0].dropna()
  return temp # 데이터 프레임에 일별 시세 저장

In [17]:
# 잘 가져와졌는지 테스트
for item_code, item_name in result:
    print(item_code, item_name)
    page_no = 1
    print(daily_market_price(item_code, 1))

413930 WON AI ESG액티브
            날짜      종가      전일비      시가      고가      저가      거래량
1   2024.09.20  9275.0  상승  140  9320.0  9320.0  9270.0    529.0
2   2024.09.19  9135.0   하락  40  9160.0  9160.0  9105.0     15.0
3   2024.09.13  9175.0   상승  60  9180.0  9180.0  9175.0      2.0
4   2024.09.12  9115.0  상승  155  9085.0  9130.0  9085.0  20713.0
5   2024.09.11  8960.0   하락  50  8990.0  9010.0  8900.0    569.0
9   2024.09.10  9010.0   하락  50  9085.0  9085.0  9010.0  18256.0
10  2024.09.09  9060.0   하락  35  8980.0  9075.0  8980.0    504.0
11  2024.09.06  9095.0   하락  95  9085.0  9150.0  9085.0    518.0
12  2024.09.05  9190.0    하락  5  9270.0  9320.0  9185.0    616.0
13  2024.09.04  9195.0  하락  350  9265.0  9315.0  9195.0    507.0
426150 WON 대한민국국고채액티브
            날짜       종가      전일비       시가       고가       저가      거래량
1   2024.09.20  56425.0   상승  15  56410.0  56440.0  56360.0  18687.0
2   2024.09.19  56410.0  하락  100  56510.0  56510.0  56245.0    912.0
3   2024.09.13  56510.0  상승  125  5

### 종목별 일별 시세 수집

In [18]:
import time

for item_code, item_name in result:
    item_list = []
    etf_before = ""
    page_no = 1

    print(item_name)
    
    # 각 종목의 전체 페이지 읽어옴
    while True:
      etf = daily_market_price(item_code, page_no)
        
      # 마지막 페이지 번호와 마지막 페이지 번호 + 1이 동일한 결과를 출력함을 이용
      if etf.equals(etf_before):
        break
      
      etf_before = etf
      item_list.append(etf)

      print(page_no)
      page_no += 1
      time.sleep(0.1) 
    
    # 데이터 프레임으로 만들어서 저장
    df = pd.concat(item_list, ignore_index = True)
    df.to_csv(f'{item_name}.csv', index = False)
    print(f'{item_name}.csv 저장 완료')

WON AI ESG액티브
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
WON AI ESG액티브.csv 저장 완료
WON 대한민국국고채액티브
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
WON 대한민국국고채액티브.csv 저장 완료
WON 미국우주항공방산
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
WON 미국우주항공방산.csv 저장 완료
WON 미국S&P500
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
WON 미국S&P500.csv 저장 완료
WON 200
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
WON 200.csv 저장 완료
WON 단기국공채액티브
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1